In [1]:
import numpy as np
from PIL import Image
from torchvision.transforms import functional as F
from pathlib import Path

In [25]:
output_dir = Path(r'./output')
image = Path(r'D:\datasets\yq-tech\data\image\390.png')
img = Image.open(image)

In [62]:
# 设置一下调整参数和输出路径
contrast_factor = [0.6, 1.5, 2, -0.6, -1.5, -2]
operation = 'adjust_contrast'
for factor in contrast_factor:
    deal_img = F.adjust_contrast(img, factor)
    output_path = output_dir.joinpath(f"{operation}_{factor}.png")
    deal_img.save(output_path)

In [74]:
brightness_factor = [0.6, 0.8, 1.2, 1.4, 1.6, 2]
operation = 'adjust_brightness'
for factor in brightness_factor:
    deal_img = F.adjust_brightness(img, factor)
    output_path = output_dir.joinpath(f"{operation}_{factor}.png")
    deal_img.save(output_path)

In [83]:
saturation_factor = [0.6, 1.5, 2, -0.6, -1.5, -2]
operation = 'adjust_saturation'
for factor in saturation_factor:
    deal_img = F.adjust_saturation(img, factor)
    output_path = output_dir.joinpath(f"{operation}_{factor}.png")
    deal_img.save(output_path)

In [6]:
mask = Path(r'18_mask.png')
m = Image.open(mask)

In [7]:
mask_data = np.asarray(m)

In [8]:
mask_data.shape

(5000, 5000)

In [6]:
new_mask_data = mask_data[:, :, :3]

In [8]:
new_mask_data.shape

(1500, 1500, 3)

In [9]:
new_mask = Image.fromarray(new_mask_data)

In [11]:
new_mask.save('mask.png')